In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10

In [3]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [4]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

In [5]:
input_path = "F:/Kaggle/Statoil/inputs/"
batch_size = 32

In [6]:
train = pd.read_json(input_path+"train.json")
target_train=train['is_iceberg']
test = pd.read_json(input_path+"test.json")

In [7]:
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])
def size(arr):     
    return float(np.sum(arr<-5))/(75*75)

In [8]:
### feat engg
train['iso1'] = train.iloc[:, 0].apply(iso)
train['iso2'] = train.iloc[:, 1].apply(iso)
test['iso1'] = test.iloc[:, 0].apply(iso)
test['iso2'] = test.iloc[:, 1].apply(iso)

# Feature engineering s1 s2 and size.
train['s1'] = train.iloc[:,5].apply(size)
train['s2'] = train.iloc[:,6].apply(size)
test['s1'] = test['iso1'].apply(size)
test['s2'] = test['iso2'].apply(size)

train['band_1'] = train['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
train['band_2'] = train['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_1'] = test['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_2'] = test['band_2'].apply(lambda x: np.array(x).reshape(75, 75))

train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')


#####process test set!
band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, 75, 75)
band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, 75, 75)
full_img_test = np.stack([band_1_test, band_2_test,(band_1_test+band_2_test)/2], axis=1)
angle_test=test['inc_angle']
size_test=test['s1']
test['is_iceberg']=0

In [9]:
train.head()

,band_1,band_2,id,inc_angle,is_iceberg,iso1,iso2,s1,s2
0,"[[-27.878361, -27.15416, -28.668615, -29.53797...","[[-27.154118, -29.537888, -31.0306, -32.190483...",dfd5f913,43.9239,0,"[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...",0.011378,0.010133
1,"[[-12.242375, -14.920305, -14.920363, -12.6663...","[[-31.506321, -27.984554, -26.645678, -23.7676...",e25388fd,38.1562,0,"[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...",0.003022,0.016356
2,"[[-24.603676, -24.603714, -24.871029, -23.1527...","[[-24.870956, -24.092632, -20.653963, -19.4110...",58b2aaa0,45.2859,1,"[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-0.0, -0.0, -0.0, -19.411043, -0.0, -0.0, -2...",0.016178,0.010667
3,"[[-22.454607, -23.082819, -23.998013, -23.9980...","[[-27.889421, -27.519794, -27.165262, -29.1035...",4cfc3a18,43.8306,0,"[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...",0.009244,0.008533
4,"[[-26.006956, -23.164886, -23.164886, -26.8911...","[[-27.206915, -30.259186, -30.259186, -23.1649...",271f93f4,35.6256,0,"[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...","[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0...",0.012089,0.007822


In [10]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [11]:
### concat angle and size
Tr_s1 = train['s1']
Tr_s2 = train['s2']
Te_s1 = test['s1']
Te_s2 = test['s2']

In [12]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [13]:
X_train = get_scaled_imgs(train)
X_test = get_scaled_imgs(test)

In [14]:
print("shape of train: {} and test: {}" .format(X_train.shape,X_test.shape))

shape of train: (1604, 75, 75, 3) and test: (8424, 75, 75, 3)


In [15]:
a= np.vstack((X_angle,Tr_s1,Tr_s2)).T
a[0].shape

(3,)

In [16]:
a

array([[  4.39239000e+01,   1.13777778e-02,   1.01333333e-02],
       [  3.81562000e+01,   3.02222222e-03,   1.63555556e-02],
       [  4.52859000e+01,   1.61777778e-02,   1.06666667e-02],
       ..., 
       [  3.52987000e+01,   1.42222222e-02,   1.20888889e-02],
       [  3.52987000e+01,   9.77777778e-03,   7.82222222e-03],
       [  3.52987000e+01,   1.03111111e-02,   1.13777778e-02]])

In [17]:
#### updated to include size and other variables

# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays

def gen_flow_for_two_inputs(X1, X2, s1,s2,y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,s1, batch_size=batch_size,seed=55)
    genX4 = gen.flow(X1,s2, batch_size=batch_size,seed=55)

    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], (np.vstack((X2i[1],X3i[1],X4i[1])).T)], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=20, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.01)
    return [es, msave,reduce_lr]

In [18]:
# # Define the image transformations here
# gen = ImageDataGenerator(horizontal_flip = True,
#                          vertical_flip = True,
#                          width_shift_range = 0.,
#                          height_shift_range = 0.,
#                          channel_shift_range=0,
#                          zoom_range = 0.2,
#                          rotation_range = 10)

# # Here is the function that merges our two generators
# # We use the exact same generator with the same random seed for both the y and angle arrays
# def gen_flow_for_two_inputs(X1, X2, y):
#     genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
#     genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
#     while True:
#             X1i = genX1.next()
#             X2i = genX2.next()
#             #Assert arrays are equal - this was for peace of mind, but slows down training
#             #np.testing.assert_array_equal(X1i[0],X2i[0])
#             yield [X1i[0], X2i[1]], X1i[1]

# # Finally create generator
# def get_callbacks(filepath, patience=2):
#     es = EarlyStopping('val_loss', patience=10, mode="min")
#     msave = ModelCheckpoint(filepath, save_best_only=True)
#     return [es, msave]

In [19]:
X_train.shape[1:]

(75, 75, 3)

In [20]:
def getVggAngleModel():
    input_2 = Input(shape=[3,], name="angle")
    angle_layer = Dense(1,activation='relu')(input_2)
    
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [21]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle,Tr_s1,Tr_s2, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle +size
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        #size
        X_size1_cv = Tr_s1[train_idx]
        X_size2_cv = Tr_s2[train_idx]
        X_size1_hold = Tr_s1[test_idx]
        X_size2_hold = Tr_s2[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug1_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv,X_size1_cv,X_size2_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout),
                callbacks=callbacks)
        

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,np.vstack((X_angle_cv,X_size1_cv,X_size2_cv)).T], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, np.vstack((X_test_angle,Te_s1,Te_s2)).T])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, np.vstack((X_angle,Tr_s1,Tr_s2)).T])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [22]:
batch_size = 32
preds=myAngleCV(X_train, X_angle, Tr_s1,Tr_s2,X_test)


===================FOLD= 0


C:\Users\ADMIN\Anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 5s 225ms/step - loss: 0.6817 - acc: 0.5781 - val_loss: 0.5674 - val_acc: 0.7671
Epoch 2/100
24/24 [==============================] - 3s 144ms/step - loss: 0.6361 - acc: 0.6234 - val_loss: 0.6902 - val_acc: 0.5311
Epoch 3/100
24/24 [==============================] - 4s 164ms/step - loss: 0.6664 - acc: 0.6055 - val_loss: 0.5659 - val_acc: 0.7733
Epoch 4/100
24/24 [==============================] - 4s 159ms/step - loss: 0.5793 - acc: 0.7261 - val_loss: 0.4216 - val_acc: 0.8075
Epoch 5/100
24/24 [==============================] - 4s 157ms/step - loss: 0.4519 - acc: 0.7721 - val_loss: 0.4468 - val_acc: 0.7795
Epoch 6/100
24/24 [==============================] - 4s 157ms/step - loss: 0.4149 - acc: 0.7786 - val_loss: 0.3181 - val_acc: 0.8385
Epoch 7/100
24/24 [==============================] - 4s 157ms/step - loss: 0.3518 - acc: 0.8357 - val_loss: 0.2821 - val_acc: 0.8571
Epoch 8/100
24/24 [==============================] - 4s 157ms/step - 

24/24 [==============================] - 4s 152ms/step - loss: 0.2820 - acc: 0.8761 - val_loss: 0.2446 - val_acc: 0.8910
Epoch 13/100
24/24 [==============================] - 4s 165ms/step - loss: 0.3108 - acc: 0.8503 - val_loss: 0.2205 - val_acc: 0.9097
Epoch 14/100
24/24 [==============================] - 4s 157ms/step - loss: 0.2300 - acc: 0.8970 - val_loss: 0.2807 - val_acc: 0.8972
Epoch 15/100
24/24 [==============================] - 4s 157ms/step - loss: 0.3144 - acc: 0.8711 - val_loss: 0.2218 - val_acc: 0.8941
Epoch 16/100
24/24 [==============================] - 4s 157ms/step - loss: 0.2734 - acc: 0.8787 - val_loss: 0.2111 - val_acc: 0.9003
Epoch 17/100
24/24 [==============================] - 4s 165ms/step - loss: 0.2568 - acc: 0.8932 - val_loss: 0.2073 - val_acc: 0.9034
Epoch 18/100
24/24 [==============================] - 4s 153ms/step - loss: 0.3648 - acc: 0.8441 - val_loss: 0.2231 - val_acc: 0.9003
Epoch 19/100
24/24 [==============================] - 4s 153ms/step - loss:

24/24 [==============================] - 4s 161ms/step - loss: 0.1840 - acc: 0.9180 - val_loss: 0.1921 - val_acc: 0.9315
Epoch 74/100
24/24 [==============================] - 4s 153ms/step - loss: 0.1541 - acc: 0.9400 - val_loss: 0.1848 - val_acc: 0.9283
Epoch 75/100
24/24 [==============================] - 4s 158ms/step - loss: 0.1471 - acc: 0.9427 - val_loss: 0.2032 - val_acc: 0.9190
Epoch 76/100
24/24 [==============================] - 4s 153ms/step - loss: 0.2445 - acc: 0.9045 - val_loss: 0.1827 - val_acc: 0.9315
Epoch 77/100
24/24 [==============================] - 4s 155ms/step - loss: 0.1678 - acc: 0.9296 - val_loss: 0.2064 - val_acc: 0.9190
Epoch 78/100
24/24 [==============================] - 4s 157ms/step - loss: 0.1632 - acc: 0.9297 - val_loss: 0.2201 - val_acc: 0.9128
Epoch 79/100
24/24 [==============================] - 4s 152ms/step - loss: 0.1723 - acc: 0.9244 - val_loss: 0.1843 - val_acc: 0.9346
Epoch 80/100
24/24 [==============================] - 4s 158ms/step - loss:

24/24 [==============================] - 4s 153ms/step - loss: 0.1896 - acc: 0.9152 - val_loss: 0.1781 - val_acc: 0.9097
Epoch 48/100
24/24 [==============================] - 4s 152ms/step - loss: 0.1746 - acc: 0.9296 - val_loss: 0.1876 - val_acc: 0.9315
Epoch 49/100
24/24 [==============================] - 4s 158ms/step - loss: 0.1877 - acc: 0.9258 - val_loss: 0.1702 - val_acc: 0.9283
Epoch 50/100
24/24 [==============================] - 4s 155ms/step - loss: 0.2647 - acc: 0.8872 - val_loss: 0.1893 - val_acc: 0.9377
Epoch 51/100
24/24 [==============================] - 4s 160ms/step - loss: 0.2127 - acc: 0.9128 - val_loss: 0.1889 - val_acc: 0.9315
Epoch 52/100
24/24 [==============================] - 4s 154ms/step - loss: 0.1795 - acc: 0.9204 - val_loss: 0.1869 - val_acc: 0.9128
Epoch 53/100
24/24 [==============================] - 4s 152ms/step - loss: 0.1983 - acc: 0.9231 - val_loss: 0.3297 - val_acc: 0.8567
Epoch 54/100
24/24 [==============================] - 4s 167ms/step - loss:

24/24 [==============================] - 4s 162ms/step - loss: 0.2117 - acc: 0.9113 - val_loss: 0.2896 - val_acc: 0.8719
Epoch 34/100
24/24 [==============================] - 4s 159ms/step - loss: 0.2269 - acc: 0.9023 - val_loss: 0.3077 - val_acc: 0.8719
Epoch 35/100
24/24 [==============================] - 4s 156ms/step - loss: 0.2161 - acc: 0.9039 - val_loss: 0.3240 - val_acc: 0.8656
Epoch 36/100
24/24 [==============================] - 4s 155ms/step - loss: 0.1923 - acc: 0.9191 - val_loss: 0.3225 - val_acc: 0.8812
Epoch 37/100
24/24 [==============================] - 4s 167ms/step - loss: 0.2286 - acc: 0.8932 - val_loss: 0.2843 - val_acc: 0.8719
Epoch 38/100
24/24 [==============================] - 4s 153ms/step - loss: 0.1939 - acc: 0.9169 - val_loss: 0.3081 - val_acc: 0.8781
Epoch 39/100
24/24 [==============================] - 4s 162ms/step - loss: 0.2169 - acc: 0.9062 - val_loss: 0.2914 - val_acc: 0.8781
Epoch 40/100
24/24 [==============================] - 4s 161ms/step - loss:

24/24 [==============================] - 4s 156ms/step - loss: 0.3024 - acc: 0.8717 - val_loss: 0.2444 - val_acc: 0.9000
Epoch 31/100
24/24 [==============================] - 4s 154ms/step - loss: 0.2045 - acc: 0.9078 - val_loss: 0.2509 - val_acc: 0.8750
Epoch 32/100
24/24 [==============================] - 4s 166ms/step - loss: 0.2153 - acc: 0.9076 - val_loss: 0.2416 - val_acc: 0.8781
Epoch 33/100
24/24 [==============================] - 4s 153ms/step - loss: 0.2491 - acc: 0.8961 - val_loss: 0.2512 - val_acc: 0.8875
Epoch 34/100
24/24 [==============================] - 4s 161ms/step - loss: 0.2251 - acc: 0.9062 - val_loss: 0.2421 - val_acc: 0.8750
Epoch 35/100
24/24 [==============================] - 4s 154ms/step - loss: 0.2217 - acc: 0.9000 - val_loss: 0.2478 - val_acc: 0.9031
Epoch 36/100
24/24 [==============================] - 4s 158ms/step - loss: 0.2124 - acc: 0.9205 - val_loss: 0.2432 - val_acc: 0.8781
Epoch 37/100
24/24 [==============================] - 4s 165ms/step - loss:

In [23]:
#Submission
import datetime as dt

submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('F:/Kaggle/Statoil/submit/sub_featengg1_5cv'+dt.datetime.today().strftime("%d%m%Y%H%M")+'.csv', index=False)